In [138]:
import yfinance as yf
import pandas as pd

In [139]:
df = yf.download("^NSEI",interval='1m',period='7d')


[*********************100%***********************]  1 of 1 completed


In [140]:
#df = df[['Close']]

In [141]:
import talib as ta

rsi = ta.RSI(df['Close'], timeperiod=14)
rsi = pd.DataFrame(rsi,columns=['rsi'])

In [142]:
"""equity = yf.Ticker("AAPL")
data = equity.history(period='1y')
df = data[['Close']]"""

'equity = yf.Ticker("AAPL")\ndata = equity.history(period=\'1y\')\ndf = data[[\'Close\']]'

In [143]:
sma = df[['Close']].rolling(window=20).mean().dropna()
rstd = df[['Close']].rolling(window=20).std().dropna()

upper_band = sma + 2 * rstd
lower_band = sma - 2 * rstd

upper_band = upper_band.rename(columns={'Close': 'upper'})
lower_band = lower_band.rename(columns={'Close': 'lower'})
bb = df.join(upper_band).join(lower_band).join(rsi)
bb = bb.dropna()

In [144]:
bb.loc[:,'idx'] = range(bb.shape[0])

In [145]:
bb.loc[:,'gain_or_loss'] = bb.Close - bb.Open
bb.loc[:,'pos_or_neg'] = bb.gain_or_loss>0

In [146]:
bb.loc[:,"above_up"] = bb.Close>bb.upper
bb.loc[:,"below_low"] = bb.Close<bb.lower

In [162]:
trade_on    = False
start_price = 0
end_price   =  0
profit=0
loss= 0 
total_trades = 0
loss_trades = 0
gain_trades = 0
profitable_trades_df = pd.DataFrame()
loss_trades_df = pd.DataFrame()
start_idx = 0
end_idx= 0 

for i,row in bb.iterrows():
    if row['above_up'] and trade_on==False:
        trade_on = True
        #print("Short selling now ",row['idx'],row['Close'])
        next_row = bb[bb.idx==row['idx']+1]
        #print(next_row['Close'][0],row['Close'])
        end_price = next_row['Open'][0]
        total_trades = total_trades+1
        bb.loc[bb['idx']==row['idx'],"trade_start"] = True
        start_idx = row['idx']
        continue
    if trade_on:
        if row['High'] > end_price+10:
            #print("Stop loss hit now",row['idx'],row['High'])
            #print("-----------------------------------------")
            trade_on = False
            start_price = 0
            end_price   =  0
            loss = loss + 10*50
            loss_trades = loss_trades+1
            bb.loc[bb['idx']==row['idx'],"trade_end"] = True
            end_idx = row['idx']
            temp = bb.loc[bb.idx.isin(range(start_idx,end_idx+1))]
            temp.loc[:,"trade_no"] = loss_trades
            
            loss_trades_df = pd.concat([loss_trades_df,temp])
            
            start_idx = 0
            end_idx= 0 
            continue
        if row['below_low']:
            start_price = row['Close']
            #print("Profit the trade is ",(end_price-start_price)*50,row['idx'])
            #print("-----------------------------------------")
            profit = profit + (end_price-start_price)*50
            trade_on = False
            gain_trades = gain_trades+1
            start_price = 0
            end_price   =  0
            bb.loc[bb['idx']==row['idx'],"trade_end"] = True
            end_idx = row['idx']
            temp = bb.loc[bb.idx.isin(range(start_idx,end_idx+1))]
            temp.loc[:,"trade_no"] = gain_trades
            profitable_trades_df = pd.concat([profitable_trades_df,temp])
            
            start_idx = 0
            end_idx= 0 
            continue
            
print("Profit for the good trades {} and number of trades taken {}".format(profit,gain_trades))
print("Loss for the bad trades {} and number of trades taken {}".format(loss,loss_trades))
print("Overall Profit for the period is",profit-loss)

/var/folders/82/55bksqyj0mb1178rtp8fv_3r0000gp/T/ipykernel_15812/3920511369.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:,"trade_no"] = loss_trades
/var/folders/82/55bksqyj0mb1178rtp8fv_3r0000gp/T/ipykernel_15812/3920511369.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:,"trade_no"] = loss_trades
/var/folders/82/55bksqyj0mb1178rtp8fv_3r0000gp/T/ipykernel_15812/3920511369.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

Profit for the good trades 31364.84375 and number of trades taken 19
Loss for the bad trades 25000 and number of trades taken 50
Overall Profit for the period is 6364.84375


/var/folders/82/55bksqyj0mb1178rtp8fv_3r0000gp/T/ipykernel_15812/3920511369.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:,"trade_no"] = gain_trades
/var/folders/82/55bksqyj0mb1178rtp8fv_3r0000gp/T/ipykernel_15812/3920511369.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:,"trade_no"] = loss_trades
/var/folders/82/55bksqyj0mb1178rtp8fv_3r0000gp/T/ipykernel_15812/3920511369.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [165]:
profitable_trades_df.to_csv("profitable_trades_df.csv")

In [166]:
loss_trades_df.to_csv("loss_trades_df.csv")

In [157]:
trade_on    = False
start_price = None
end_price   =  None
profit=0
loss= 0 
total_trades = 0
loss_trades = 0
gain_trades = 0
for i,row in bb.iterrows():
    if row['below_low'] and trade_on==False:
        trade_on = True
        #print("Short selling now ",row['idx'],row['Close'])
        next_row = bb[bb.idx==row['idx']+1]
        start_price = next_row['Open'][0]
        continue
    if trade_on:
        if row['Low'] < start_price-10:
            #print("Stop loss hit now",row['idx'],row['High'])
            #print("-----------------------------------------")
            trade_on = False
            start_price = None
            end_price   =  None
            loss = loss + 10*50
            loss_trades = loss_trades+1
            continue
        if row['above_up']:
            end_price = row['Close']
            #print("Profit the trade is ",(end_price-start_price)*50)
            #print("-----------------------------------------")
            gain_trades = gain_trades+1
            profit = profit + (end_price-start_price)*50
            trade_on = False
            continue
            
print("Profit for the good trades {} and number of trades taken {}".format(profit,gain_trades))
print("Loss for the bad trades {} and number of trades taken {}".format(loss,loss_trades))
print("Overall Profit for the period is",profit-loss)

Profit for the good trades 28842.67578125 and number of trades taken 16
Loss for the bad trades 19500 and number of trades taken 39
Overall Profit for the period is 9342.67578125


In [154]:
bb.to_csv("1min.csv")

In [ ]:
buyers = bb[bb['Close'] <= bb['lower']]
sellers = bb[bb['Close'] >= bb['upper']]

In [ ]:
bb.loc[:,'sma_20'] = sma.Close

In [14]:
sma.shape

(2605, 1)

In [15]:
bb.shape

(2605, 3)

In [ ]:
# Plotting

import plotly.io as pio
import plotly.graph_objects as go

pio.templates.default = "plotly_dark"

fig = go.Figure()
fig.add_trace(go.Scatter(x=lower_band.index, 
                         y=lower_band['lower'], 
                         name='Lower Band', 
                         line_color='rgba(173,204,255,0.2)'
                        ))
fig.add_trace(go.Scatter(x=upper_band.index, 
                         y=upper_band['upper'], 
                         name='Upper Band', 
                         fill='tonexty', 
                         fillcolor='rgba(173,204,255,0.2)', 
                         line_color='rgba(173,204,255,0.2)'
                        ))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['Close'], 
                         name='Close', 
                         line_color='#636EFA'
                        ))
fig.add_trace(go.Scatter(x=sma.index, 
                         y=sma['Close'], 
                         name='SMA', 
                         line_color='#FECB52'
                        ))
fig.add_trace(go.Scatter(x=buyers.index, 
                         y=buyers['Close'], 
                         name='Buyers', 
                         mode='markers',
                         marker=dict(
                             color='#00CC96',
                             size=10,
                             )
                         ))
fig.add_trace(go.Scatter(x=sellers.index, 
                         y=sellers['Close'], 
                         name='Sellers', 
                         mode='markers', 
                         marker=dict(
                             color='#EF553B',
                             size=10,
                             )
                         ))
fig.show()

In [ ]:
bb

In [167]:
df = yf.download("^NSEI",interval='1m',period='1d')


[*********************100%***********************]  1 of 1 completed


In [168]:
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-09-08 09:15:00+05:30,17748.150391,17765.099609,17746.699219,17752.750000,17752.750000,0
2022-09-08 09:16:00+05:30,17751.050781,17760.150391,17746.650391,17757.150391,17757.150391,0
2022-09-08 09:17:00+05:30,17757.500000,17760.650391,17744.800781,17747.000000,17747.000000,0
2022-09-08 09:18:00+05:30,17744.250000,17752.599609,17741.300781,17741.550781,17741.550781,0
2022-09-08 09:19:00+05:30,17742.400391,17744.699219,17732.800781,17733.900391,17733.900391,0
...,...,...,...,...,...,...
2022-09-08 15:26:00+05:30,17799.000000,17801.500000,17798.699219,17800.849609,17800.849609,0
2022-09-08 15:27:00+05:30,17800.699219,17803.849609,17799.300781,17801.449219,17801.449219,0
2022-09-08 15:28:00+05:30,17801.150391,17804.500000,17799.650391,17801.800781,17801.800781,0
